Modele takie jak maxvit_t, swin_t czy vit_b_16 są dostępne dopiero od nowszych wersji torchvision (od wersji 0.13 lub 0.14). Jeśli użyjesz wcześniejszych wersji, pojawią się błędy, np. AttributeError: module torchvision.models has no attribute maxvit_t.
Sugestia: Sprawdź wersję biblioteki torchvision w swoim środowisku:
import torchvision
print(torchvision.__version__)
Jeśli używasz starszej wersji, zaktualizuj ją:
pip install --upgrade torchvision

In [ ]:
import torch
import multiModel as mm
from multiModel import MultiInputModel
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
import os
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

#Zapisz wynikii w 
path_results = "training_results2"
# Włącz blokowanie błędów CUDA
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Załaduj dane
train_dataset = mm.MultiInputDataset("CSV/dataset/train.csv", transform_rgb=mm.transform_rgb, transform_binary=mm.transform_binary)
val_dataset = mm.MultiInputDataset("CSV/dataset/val.csv", transform_rgb=mm.transform_rgb, transform_binary=mm.transform_binary)
test_dataset = mm.MultiInputDataset("CSV/dataset/test.csv", transform_rgb=mm.transform_rgb, transform_binary=mm.transform_binary)

# List of models to train
models_list = ['efficientnet_b0','efficientnet_v2_m', 'mobilenet_v3_small', 'resnet34','swin_t', 'vit_b_16']#
batch_sizes = {
    'efficientnet_b0': 32,
    'mobilenet_v3_small': 32,
    'efficientnet_v2_m': 16,
    'resnet34': 32,
    'swin_t' :32,
    'vit_b_16': 16  
}
for model_name in models_list:
    # Inicjalizacja modelu
    model = MultiInputModel(num_classes=11, base_model=model_name)  # Liczba klas
    model = model.to("cuda")  # Jeśli używasz GPU

    #Dynamicznie przydzielany batch_size

    batch_size = batch_sizes[model_name]
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    # Optymalizator i funkcja straty
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Zapis logów
    log_file = f"{path_results}/training_log_{model_name}.txt"
    with open(log_file, "w") as f:
        f.write("epoch,train_loss,val_loss,train_accuracy,val_accuracy,train_precision,val_precision,train_recall,val_recall,train_f1,val_f1\n")

    # Wczesne zatrzymanie - parametry
    early_stop_patience = 5  # Liczba epok bez poprawy
    best_val_loss = float("inf")
    patience_counter = 0
    best_model_path = f"{path_results}/best_model_{model_name}.pth"

    # Pętla treningowa
    num_epochs = 50
    for epoch in range(num_epochs):
        # === TRENING ===
        print(f"Epoch {epoch + 1}/{num_epochs}")
        model.train()
        train_loss = 0
        train_true = []
        train_pred = []

        # Dodaj pasek postępu do pętli batchy
        with tqdm(total=len(train_loader), desc="Training", unit="batch") as pbar:
            for t_image, b_image, s_image, labels in train_loader:
                t_image, b_image, s_image, labels = (
                    t_image.to("cuda"),
                    b_image.to("cuda"),
                    s_image.to("cuda"),
                    labels.to("cuda")
                )

                # Oblicz predykcje i stratę
                outputs = model(t_image, b_image, s_image)
                loss = criterion(outputs, labels)

                # Backpropagation
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                train_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                train_true.extend(labels.cpu().numpy())
                train_pred.extend(predicted.cpu().numpy())
                pbar.set_postfix({"loss": f"{train_loss / (pbar.n + 1):.4f}"})  # Wyświetl średnią stratę
                pbar.update(1)  # Aktualizuj pasek postępu o 1 krok

        train_loss /= len(train_loader)  # Średnia strata w treningu
        train_accuracy = accuracy_score(train_true, train_pred)
        train_precision = precision_score(train_true, train_pred, average="weighted")
        train_recall = recall_score(train_true, train_pred, average="weighted")
        train_f1 = f1_score(train_true, train_pred, average="weighted")
        #print(f"Train Loss: {train_loss:.4f}")

        # === WALIDACJA ===
        model.eval()
        val_loss = 0
        val_true = []
        val_pred = []

        with torch.no_grad():  # Wyłącz gradienty
            with tqdm(total=len(val_loader), desc="Validation", unit="batch") as pbar_val:
                for t_image, b_image, s_image, labels in val_loader:
                    t_image, b_image, s_image, labels = (
                        t_image.to("cuda"),
                        b_image.to("cuda"),
                        s_image.to("cuda"),
                        labels.to("cuda")
                    )
                    outputs = model(t_image, b_image, s_image)
                    loss = criterion(outputs, labels)
                    val_loss += loss.item()

                    _, predicted = torch.max(outputs, 1)
                    val_true.extend(labels.cpu().numpy())
                    val_pred.extend(predicted.cpu().numpy())

                    pbar_val.update(1)  # Aktualizuj pasek postępu walidacji

        val_loss /= len(val_loader)  # Średnia strata w walidacji
        val_accuracy = accuracy_score(val_true, val_pred)
        val_precision = precision_score(val_true, val_pred, average="weighted")
        val_recall = recall_score(val_true, val_pred, average="weighted")
        val_f1 = f1_score(val_true, val_pred, average="weighted")
        #print(f"Val Loss: {val_loss:.4f}")

        # === LOGI ===
        #print(f"Epoch {epoch + 1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
        with open(log_file, "a") as f:
            f.write(
                f"{epoch + 1},{train_loss:.4f},{val_loss:.4f},{train_accuracy:.4f},{val_accuracy:.4f},"
                f"{train_precision:.4f},{val_precision:.4f},{train_recall:.4f},{val_recall:.4f},{train_f1:.4f},{val_f1:.4f}\n"
            )

        # === WCZESNE ZATRZYMANIE ===
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            # Zapis najlepszego modelu
            torch.save(model, best_model_path) #Zapisanie modelu i architektury w pliku pth
            print(f"Best model saved at epoch {epoch + 1}")
        else:
            patience_counter += 1
            print(f"No improvement in val loss for {patience_counter} epoch(s)")

        if patience_counter >= early_stop_patience:
            print("Early stopping triggered. Training stopped.")
            break
    """
    # === TEST ===
    # Wczytaj najlepszy model
    model.load(torch.load(best_model_path))
    model.eval()

    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for t_image, b_image, s_image, labels in test_loader:
            t_image, b_image, s_image, labels = (
                t_image.to("cuda"),
                b_image.to("cuda"),
                s_image.to("cuda"),
                labels.to("cuda")
            )
            outputs = model(t_image, b_image, s_image)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            # Oblicz dokładność
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    test_loss /= len(test_loader)
    accuracy = correct / total

    # Zapis wyniku testu
    log_file_test = f"training_results/test_log_{model_name}.txt"
    with open(log_file_test, "w") as f_t:
            f_t.write(f"Test Loss: {test_loss:.4f}, Test Accuracy: {accuracy:.4f}")
    """
    # Zwalnianie pamięci po zakończeniu pracy z modelem
    del model  # Usuń model z pamięci
    torch.cuda.empty_cache()  # Wyczyść pamięć GPU


/home/mk/miniconda3/envs/dnn_gpu/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/mk/miniconda3/envs/dnn_gpu/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model: efficientnet_b0, base_model_output_size: 1280
Total input size to fc: 2688
Epoch 1/50


Training:   0%|          | 1/2172 [00:01<48:39,  1.34s/batch, loss=2.4073]

In [ ]:
# === TEST ===
# Wczytaj najlepszy model
model = torch.load(best_model_path)
model.eval()

test_loss = 0
correct = 0
total = 0
with torch.no_grad():
    for t_image, b_image, s_image, labels in test_loader:
        t_image, b_image, s_image, labels = (
            t_image.to("cuda"),
            b_image.to("cuda"),
            s_image.to("cuda"),
            labels.to("cuda")
        )
        outputs = model(t_image, b_image, s_image)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # Oblicz dokładność
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

test_loss /= len(test_loader)
accuracy = correct / total

# Zapis wyniku testu
log_file_test = f"training_results/test_log_{model_name}.txt"
with open(log_file_test, "w") as f_t:
        f_t.write(f"Test Loss: {test_loss:.4f}, Test Accuracy: {accuracy:.4f}")

# Zwalnianie pamięci po zakończeniu pracy z modelem
del model  # Usuń model z pamięci
torch.cuda.empty_cache()  # Wyczyść pamięć GPU


*******************